# MIMIC-III query API tutorial

This notebook shows examples of how to use the cyclops.query API on [MIMIC-III v1.4](https://physionet.org/content/mimiciii/1.4/).

Each query is limit to 100 rows (for quick results).

* First, setup the MIMIC-III database according to the instructions in [mimic-code](https://github.com/MIT-LCP/mimic-code/tree/main/mimic-iii/buildmimic/postgres).
* The database is assumed to be hosted using postgres. Update the config parameters such as username and password, passed to `MIMICIIIQuerier` accordingly.

## Imports and instantiate `MIMICIIIQuerier`

In [ ]:
"""MIMICIII query API tutorial."""

import cyclops.query.ops as qo
from cyclops.query import MIMICIIIQuerier


querier = MIMICIIIQuerier(
    dbms="postgresql",
    port=5432,
    host="localhost",
    database="mimiciii",
    user="postgres",
    password="pwd",
)
# List all custom table methods.
querier.list_custom_tables()

## Example 1. Get all male patients with a mortality outcome.

In [ ]:
ops = qo.Sequential(
    qo.ConditionEquals("expire_flag", 1),
    qo.ConditionEquals("gender", "M"),
)
patients = querier.mimiciii.patients()
patients = patients.ops(ops).run(limit=100)
print(f"{len(patients)} rows extracted!")

## Example 2. Get all female patient encounters with diagnoses (`gastroenteritis` in ICD-9 long title).

In [ ]:
patients = querier.mimiciii.patients()
patients = patients.ops(qo.ConditionEquals("gender", "F"))
admissions = querier.mimiciii.admissions()
patient_admissions = patients.join(
    join_table=admissions,
    on="subject_id",
)
diagnoses = querier.diagnoses()
diagnoses = diagnoses.ops(qo.ConditionSubstring("long_title", "gastroenteritis"))
patient_admissions_diagnoses = patient_admissions.join(
    join_table=diagnoses,
    on=["subject_id", "hadm_id"],
).run(limit=100)
print(f"{len(patient_admissions_diagnoses)} rows extracted!")

## Example 3. Get potassium lab tests for female patients.

In [ ]:
patients = querier.mimiciii.patients()
patients = patients.ops(qo.ConditionEquals("gender", "F"))
labs = querier.labevents()
labs = labs.ops(qo.ConditionEquals("label", "potassium"))
patient_labs = patients.join(labs, on="subject_id").run(limit=100)
print(f"{len(patient_labs)} rows extracted!")

## Example 4. Get AaDO2 carevue chart events for male patients that have a `valuenum` of less than 20.

In [ ]:
chartevents_ops = qo.Sequential(
    qo.ConditionEquals("dbsource", "carevue"),
    qo.ConditionEquals("label", "AaDO2"),
    qo.ConditionLessThan("valuenum", 20),
)
patients = querier.mimiciii.patients()
patients = patients.ops(qo.ConditionEquals("gender", "M"))
chart_events = querier.chartevents()
chart_events = chart_events.ops(chartevents_ops)
patient_chart_events = patients.join(chart_events, on="subject_id").run(limit=100)
print(f"{len(patient_chart_events)} rows extracted!")